### Setup and data loading

In [95]:
from pathlib import Path
import joblib

import numpy as np
import polars as pl

from rlbidder.viz import (
    plot_campaign_simulation_metrics,
    plot_score_distribution_by_agent,
    plot_score_by_agent,
    plot_interval_estimates,
    plot_budget_pacing_trajectories,
    plot_auction_market_overview,
    compute_agent_interval_estimates,
    compute_auction_budget_pacing,
)
from rlbidder.evaluation import summarize_agent_scores


df = pl.read_parquet([
    "eval/df_agent_summaries_FixedCPA_nockpt.parquet",
    "eval/df_agent_summaries_StochasticCPA_nockpt.parquet", 
    "eval/df_agent_summaries_ValueScaledCPA_nockpt.parquet", 
    "eval/df_agent_summaries_BudgetPacer_nockpt.parquet", 
    "eval/df_agent_summaries_PIDBudgetPacer_nockpt.parquet", 
    "eval/df_agent_summaries_PIDCPA_nockpt.parquet", 
    "eval/df_agent_summaries_BC_best.parquet", 
    "eval/df_agent_summaries_CQL_best.parquet",
    "eval/df_agent_summaries_IQL_best.parquet",
    "eval/df_agent_summaries_DT_best.parquet",
    "eval/df_agent_summaries_GAVE_best.parquet",
    "eval/df_agent_summaries_GAS_best.parquet",
    
])
df_summary, df_best_per_rotation = summarize_agent_scores(
    df,
    group_by_period=True, 
    return_intermediate=True,
    use_best_advertiser=False,
    filter_env_agents=True,
)
df_summary = df_summary.sort(["period", "mean_score"], descending=True)
pl.Config.set_tbl_rows(50)
df_summary


period,agent_name,mean_returns,mean_score,mean_budget_spent_ratio,mean_cpa_exceed_ratio
i64,str,f64,f64,f64,f64
17,"""IQL""",35.534722,28.953428,0.963252,0.064286
17,"""GAVE""",31.881944,27.803564,0.833953,-0.032459
17,"""CQL""",33.798611,27.563276,0.943924,0.054357
17,"""DT""",30.548611,26.697888,0.791513,-0.038754
17,"""BudgetPacer""",29.048611,26.538538,0.689401,-0.098427
17,"""GAS""",30.888889,26.1435,0.815247,-0.035632
17,"""PIDCPA""",25.451389,24.476407,0.734288,-0.08852
17,"""StochasticCPA""",25.416667,24.36937,0.733834,-0.085771
17,"""FixedCPA""",25.354167,24.331319,0.734002,-0.086271


In [72]:
agent_name = "IQL"
pt = "best"

auction_histories = joblib.load(f"eval/auction_histories_{agent_name}_{pt}.joblib")
auction_histories[:2]

[StepwiseAuctionHistory(steps_added=48, num_advertisers=48, ),
 StepwiseAuctionHistory(steps_added=48, num_advertisers=48, )]

### Market overview

In [73]:
auction_idx = 0

fig = plot_auction_market_overview(
    auction_histories[auction_idx],
    bid_threshold=0.01,
    gap_threshold=0.0,
    win_threshold=0.1,
    pct=0.05,
    vol_window=4,
    use_log=True,
)
fig.show()

### Agent Campaign

In [74]:
history_idx = 5

agent_indices = np.where(auction_histories[history_idx].agent_names == agent_name)[0]
agent_idx = agent_indices[1]

fig, scatter_fig = plot_campaign_simulation_metrics(auction_histories[history_idx], agent_idx)
fig.show()
scatter_fig.show()

### Agent Budget Pacing

In [75]:
fig = plot_budget_pacing_trajectories(
    *compute_auction_budget_pacing(
        auction_histories, 
        agent_name=agent_name, 
        sample_size=300, 
        seed=42,
    ),
    agent_name=agent_name,
)
fig.show()


### Performance Benchmark

In [ ]:
fig = plot_score_distribution_by_agent(
    df_best_per_rotation,
    agent_col="agent_name",
    score_col="mean_score",
    relative=True,
    quantile=0.05,
)
fig.show()

fig = plot_score_by_agent(
    df_best_per_rotation, 
    agent_col="agent_name",
    score_col="mean_score",
    statistic="median",
    relative=True, 
    quantile=0.05,
)
fig.show()

In [94]:
aggregate_scores, aggregate_score_cis = compute_agent_interval_estimates(
    df_best_per_rotation,
    score_col="mean_score",
    run_setting_cols=["period", "rotate_index"],
    normalize_scores=True,
    reps=20000,
    task_bootstrap=True,
)
fig = plot_interval_estimates(
    point_estimates=aggregate_scores,
    interval_estimates=aggregate_score_cis,
    metric_names=['Mean', 'Median', 'IQM', 'Optimality Gap'],
    xlabel='Evaluation Score',
    fig_width=1200,
)
fig.show()

100%|██████████| 12/12 [00:36<00:00,  3.04s/it]
